# Virtual Arena

This notebook is an implementation of a virtual arena using CoppeliaSim.

---

In [1]:
# Importing modules...
import numpy as np
import scipy as sp
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.synchronizer import Synchronizer

from modules.integration.client import Client
from modules.integration.coppeliasim.server import CoppeliaSim_Server
from modules.integration.coppeliasim.camera import CoppeliaSim_Camera

# Setting the Scene

All `Camera` and `Client` objects will be instanciated right along with the `Server` object. A CoppeliaSim scene will be requested given the arbitrary camera parameters.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

# Create clients
for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    pose = np.vstack((R @ base_matrix,
                      np.array([0, 0, 0, 1])))

    # Generate associated camera model
    camera = (CoppeliaSim_Camera(resolution=(1080, 1080), 
                                 fov_degrees=60.0,     
                                 pose=pose,
                                 distortion_model='fisheye',
                                 distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
                                 snr_dB=13
                                 ))
    
    clients.append(Client(camera=camera))

# Create server
server = CoppeliaSim_Server(clients=clients,
                            server_address=('127.0.0.1', 8888),
                            controller_address=('127.0.0.1', 7777))

# Request scene with the associated server clients
if not server.request_scene():
    sys.exit() # Scene request failed!

[SERVER] Wrapping up CoppeliaSim scene info
[SERVER] Scene info sent
[SERVER] Scene set!


# Extrinsic Calibration

The Cameras' Extrinsic Parameters will be estimated. For that, a calibration routine will be requested and the data will be post-processed for the parameter estimation.

---

In [3]:
# Capture specifications
blob_count = 3 # Number of expected markers
capture_time = 30.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_calibration(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Extrinsic Calibration info sent
[SERVER] Extrinsic Calibration confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


In [4]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Receiving messages
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

# Post-processing
for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 
        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


In [5]:
# Calibrate multiple view
wand_distances = np.array([5e-2, 10e-2, 15e-2]) # In meters

wand_blobs = [client.synchronizer.sync_blobs for client in server.clients]

if not server.multiple_view.calibrate(wand_blobs, wand_distances):
    sys.exit() # Calibration failed!

In [6]:
# Create the Scene Viewer
scene = Viewer3D(title='Calibrated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

In [7]:
# Perform bundle adjustment
n_observations = 72 # Choose a number multiple of the total number of unique pairs: n_cameras * (n_cameras - 1) / 2
server.multiple_view.bundle_adjustment(wand_blobs, wand_distances, n_observations)

In [8]:
# Create the Scene Viewer
scene = Viewer3D(title='Adjusted Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Reference Update

Right after the Extrinsic Calibration, a new reference will be set to be the new scene's canonical frame. For that, a new reference will be requested.

---

In [9]:
# Capture specifications
blob_count = 3 # Number of expected markers
capture_time = 1.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_reference(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Reference Update info sent
[SERVER] Reference Update confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


In [10]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 
        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


In [11]:
# Measured distances between perpendicularly matched marker distances
# Distances: [D_x, D_y]
wand_distances = np.array([7.5e-2, 15e-2]) # In meters

# Triangulation pair
pair = (0, 2) # Diagonal pairs seems to produce more stable results

wand_blobs = [server.clients[ID].synchronizer.async_blobs for ID in pair]

# Update reference
server.multiple_view.update_reference(wand_blobs, wand_distances, pair)

In [12]:
# Create the Scene Viewer
scene = Viewer3D(title='Updated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Add new reference
scene.add_frame(np.eye(4), 'Reference', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Standard Capture

With all calibration done, a standard capture routine can be requested for the arena's usual operation.

---

In [23]:
# Capture specifications
blob_count = 4 # Number of expected markers
capture_time = 10.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Capture info sent
[SERVER] Parsing failed!


SystemExit: 

In [ ]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

triangulation = 0
sync_triangulated_markers = np.empty((server.clients[0].synchronizer.sync_PTS.size, blob_count, 3))
visualizer_address = ('127.0.0.1', 6666)

# Breaks in the timeout
while True:
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]}):')

    # Decode message
    try:
        message = np.frombuffer(message_bytes, dtype=np.float32)

    except:
        if verbose: print('> Couldn\'t decode message')

        continue # Jump to wait for the next message

    # Empty message
    if not message.size:
        if verbose: print('\tEmpty message')

        continue # Jump to wait for the next message

    # Extracting the message's PTS
    PTS = message[-1] # Last element of the message 

    # Valid message is [u, v, A] per blob and the PTS of the message
    if message.size !=  3 * blob_count + 1:

        if message.size == 1: # Only PTS
            if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

        else: 
            if verbose: 
                print(f'\tWrong blob count or corrupted message')
                print(f'Corrupted Message: {message}')

        continue # Jump to wait for the next message

    # Extracting blob data (coordinates & area)
    blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

    # Extracting centroids
    blob_centroids = blob_data[:,:2] # Ignoring their area

    # Undistorting blobs centroids
    undistorted_blobs = server.clients[ID].camera.undistort_points(blob_centroids)          

    # Print blobs
    if verbose:
        print(f'\tDetected Blobs - {PTS :.3f} s')
        print('\t' + str(blob_data).replace('\n', '\n\t'))

    # Save data
    valid_data = server.clients[ID].synchronizer.add_data(undistorted_blobs, PTS)

    if verbose: 
        if valid_data:
            print('\tData Accepted!')
        else:
            print('\tData Refused!')

    # Check for available interpolated data
    available = []

    synchronizers = [c.synchronizer for c in server.clients]
    for ID, S in enumerate(synchronizers):
        # Is there interpolated data? Non-interpolated blobs are negative!
        if np.any(S.sync_blobs[triangulation] >= 0): 
            available.append(ID)

    # If at least two cameras have interpolated data, triangulate 
    if len(available) >= 2:
        # Get the first two available
        pair = available[:2]
        reference, auxiliary = pair

        blobs_pair = [synchronizers[reference].sync_blobs[triangulation],
                      synchronizers[auxiliary].sync_blobs[triangulation]]
        
        triangulated_markers = server.multiple_view.triangulate_by_pair(pair, blobs_pair)

        # Triangulation is not reliable
        if np.isnan(triangulated_markers).any():
            # Update last triangulation flag
            if triangulation < sync_triangulated_markers.shape[0] - 1:
                triangulation += 1

            continue # Jump to wait for the next message

        # Resending to CoppeliaSim
        buffer = triangulated_markers.T.astype(np.float32).ravel().tobytes()
        server.udp_socket.sendto(buffer, visualizer_address)

        # Update last triangulation flag
        if triangulation < sync_triangulated_markers.shape[0] - 1:
            triangulation += 1

[SERVER] Timeout set to 5 seconds



ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto